In [2]:
%pip install pulp
%pip install pyomo
%pip install plotly
%pip install statsmodels

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append('..')
from src import feature_engineering

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Feature Engineering cho Dự báo Nhu cầu

Notebook này thực hiện việc tạo đặc trưng (feature engineering) cho mô hình dự báo nhu cầu, bao gồm:

1. **Time-based Features**:
   - Lag features (nhu cầu các tuần trước)
   - Rolling statistics (trung bình động, độ lệch chuẩn)
   - Các đặc trưng theo chu kỳ (tuần, ngày)

2. **Product Features**:
   - Đặc trưng theo department/aisle
   - Thống kê tái mua
   - Phân loại sản phẩm

3. **Customer Features**:
   - Hành vi mua sắm
   - Tần suất mua hàng
   - Độ trung thành với sản phẩm

In [5]:
import os

# Load dữ liệu từ kết quả EDA
DATA_DIR = 'C:/Users/binhd/Downloads/New folder/data'

aisles = pd.read_csv(os.path.join(DATA_DIR, 'aisles.csv'))
departments = pd.read_csv(os.path.join(DATA_DIR, 'departments.csv'))
order_prior = pd.read_csv(os.path.join(DATA_DIR, 'order_products__prior.csv'))
orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv'))
products = pd.read_csv(os.path.join(DATA_DIR, 'products.csv'))

# Merge dữ liệu
df = order_prior.merge(orders, on='order_id', how='left')\
                .merge(products, on='product_id', how='left')\
                .merge(aisles, on='aisle_id', how='left')\
                .merge(departments, on='department_id', how='left')

print("\nShape của DataFrame sau khi merge:", df.shape)
print("\nCác cột trong DataFrame:")
print(df.columns.tolist())


Shape của DataFrame sau khi merge: (32434489, 15)

Các cột trong DataFrame:
['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'product_name', 'aisle_id', 'department_id', 'aisle', 'department']


# 1. Time-based Features

Tạo các đặc trưng liên quan đến thời gian:

1. **Lag Features**:
   - Nhu cầu của 1-4 tuần trước
   - Tỷ lệ tái mua các tuần trước
   
2. **Rolling Features**:
   - Trung bình động (2 và 4 tuần)
   - Độ lệch chuẩn động
   
3. **Cyclical Features**:
   - Ngày trong tuần
   - Giờ trong ngày
   - Cuối tuần/ngày thường

In [6]:
# Tính tuần dựa trên order_number
df['week'] = df['order_number'].map(lambda x: (x-1) // 7 + 1)

# Tạo demand_df ban đầu
demand_df = df.groupby(['product_id', 'week']).agg({
    'order_id': 'count',  # số lượng đơn hàng
    'reordered': ['sum', 'mean'],  # số lượng và tỷ lệ tái mua
    'department_id': 'first',
    'aisle_id': 'first',
    'product_name': 'first'
}).reset_index()

# Đổi tên cột
demand_df.columns = ['product_id', 'week', 'demand', 'reorder_count', 'reorder_rate', 
                    'department_id', 'aisle_id', 'product_name']

print("Shape của demand_df ban đầu:", demand_df.shape)

# 1. Tạo lag features
for lag in [1, 2, 3, 4]:
    demand_df[f'demand_lag_{lag}'] = demand_df.groupby('product_id')['demand'].shift(lag)
    demand_df[f'reorder_rate_lag_{lag}'] = demand_df.groupby('product_id')['reorder_rate'].shift(lag)

# 2. Tạo rolling features
for window in [2, 4]:
    # Rolling mean
    demand_df[f'demand_rolling_mean_{window}'] = demand_df.groupby('product_id')['demand']\
        .rolling(window=window).mean().reset_index(0, drop=True)
    demand_df[f'reorder_rate_rolling_mean_{window}'] = demand_df.groupby('product_id')['reorder_rate']\
        .rolling(window=window).mean().reset_index(0, drop=True)
    
    # Rolling std
    demand_df[f'demand_rolling_std_{window}'] = demand_df.groupby('product_id')['demand']\
        .rolling(window=window).std().reset_index(0, drop=True)

# 3. Tạo cyclical features
demand_df['day_type'] = demand_df['week'].mod(7)  # ngày trong tuần
demand_df['is_weekend'] = demand_df['day_type'].isin([5, 6]).astype(int)

print("\nCác features đã tạo:")
time_features = [col for col in demand_df.columns if any(x in col for x in ['lag', 'rolling', 'day', 'weekend'])]
print("\n".join(time_features))

Shape của demand_df ban đầu: (413313, 8)

Các features đã tạo:
demand_lag_1
reorder_rate_lag_1
demand_lag_2
reorder_rate_lag_2
demand_lag_3
reorder_rate_lag_3
demand_lag_4
reorder_rate_lag_4
demand_rolling_mean_2
reorder_rate_rolling_mean_2
demand_rolling_std_2
demand_rolling_mean_4
reorder_rate_rolling_mean_4
demand_rolling_std_4
day_type
is_weekend


# 2. Product Features

Tạo các đặc trưng liên quan đến sản phẩm:

1. **Department/Aisle Features**:
   - Thống kê nhu cầu theo department
   - Thống kê nhu cầu theo aisle
   
2. **Reorder Features**:
   - Tỷ lệ tái mua trung bình
   - Khoảng cách giữa các lần mua
   
3. **Product Popularity**:
   - Xếp hạng sản phẩm theo doanh số
   - Phân loại sản phẩm theo mức độ phổ biến

In [7]:
# 1. Department/Aisle statistics
dept_stats = demand_df.groupby('department_id').agg({
    'demand': ['mean', 'std', 'max'],
    'reorder_rate': 'mean'
}).reset_index()

dept_stats.columns = ['department_id'] + [f'dept_{col[0]}_{col[1]}' for col in dept_stats.columns[1:]]

aisle_stats = demand_df.groupby('aisle_id').agg({
    'demand': ['mean', 'std', 'max'],
    'reorder_rate': 'mean'
}).reset_index()

aisle_stats.columns = ['aisle_id'] + [f'aisle_{col[0]}_{col[1]}' for col in aisle_stats.columns[1:]]

# Merge với demand_df
demand_df = demand_df.merge(dept_stats, on='department_id', how='left')
demand_df = demand_df.merge(aisle_stats, on='aisle_id', how='left')

# 2. Product popularity features
product_popularity = demand_df.groupby('product_id')['demand'].agg(['mean', 'sum']).reset_index()
product_popularity['popularity_rank'] = product_popularity['sum'].rank(method='dense', ascending=False)
product_popularity['popularity_percentile'] = product_popularity['sum'].rank(pct=True)

# Phân loại sản phẩm theo mức độ phổ biến
def get_popularity_category(percentile):
    if percentile <= 0.2:
        return 'Low'
    elif percentile <= 0.8:
        return 'Medium'
    else:
        return 'High'

product_popularity['popularity_category'] = product_popularity['popularity_percentile'].map(get_popularity_category)

# Merge với demand_df
demand_df = demand_df.merge(product_popularity.drop('sum', axis=1), on='product_id', how='left')

print("Các product features đã tạo:")
product_features = [col for col in demand_df.columns if any(x in col for x in ['dept_', 'aisle_', 'popularity'])]
print("\n".join(product_features))

Các product features đã tạo:
aisle_id
dept_demand_mean
dept_demand_std
dept_demand_max
dept_reorder_rate_mean
aisle_demand_mean
aisle_demand_std
aisle_demand_max
aisle_reorder_rate_mean
popularity_rank
popularity_percentile
popularity_category


In [8]:
# Chuẩn hóa các features số
numeric_features = demand_df.select_dtypes(include=['float64', 'int64']).columns
categorical_features = ['product_id', 'department_id', 'aisle_id', 'week', 'day_type', 
                       'is_weekend', 'popularity_category']
numeric_features = [col for col in numeric_features if col not in categorical_features]

scaler = StandardScaler()
demand_df[numeric_features] = scaler.fit_transform(demand_df[numeric_features])

# Xóa các dòng có giá trị NaN (do tạo lag features)
demand_df = demand_df.dropna()

print("Shape cuối cùng của demand_df:", demand_df.shape)
print("\nPhân bố các loại features:")
print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

# Lưu kết quả
output_path = os.path.join(DATA_DIR, 'demand_data.csv')
demand_df.to_csv(output_path, index=False)
print(f"\nĐã lưu demand_data.csv tại: {output_path}")

# Hiển thị một số dòng mẫu
print("\nMẫu dữ liệu sau khi tạo features:")
print(demand_df.head())

Shape cuối cùng của demand_df: (225537, 36)

Phân bố các loại features:
Numeric features: 28
Categorical features: 7

Đã lưu demand_data.csv tại: C:/Users/binhd/Downloads/New folder/data\demand_data.csv

Mẫu dữ liệu sau khi tạo features:
   product_id  week    demand  reorder_count  reorder_rate  department_id  \
4           1     5  0.067942       0.120845      0.847381             19   
5           1     6  0.007286       0.037262      0.708114             19   
6           1     7 -0.023042       0.017361      1.069456             19   
7           1     8 -0.056007      -0.036371      0.719331             19   
8           1     9 -0.066556      -0.044331      0.943676             19   

   aisle_id                product_name  demand_lag_1  reorder_rate_lag_1  \
4        61  Chocolate Sandwich Cookies      0.127554            0.967165   
5        61  Chocolate Sandwich Cookies      0.050817            0.874723   
6        61  Chocolate Sandwich Cookies     -0.006117            0.7

# Kết luận

Features đã tạo được chia thành các nhóm:

1. **Time-based Features**:
   - Lag features: demand_lag_1 đến demand_lag_4
   - Rolling features: demand_rolling_mean_2, demand_rolling_mean_4
   - Cyclical features: day_type, is_weekend

2. **Product Features**:
   - Department stats: dept_demand_mean, dept_demand_std, ...
   - Aisle stats: aisle_demand_mean, aisle_demand_std, ...
   - Popularity: popularity_rank, popularity_category

Các features này sẽ được sử dụng trong notebook tiếp theo để huấn luyện mô hình dự báo.